In [1]:
# -*- coding: utf-8 -*-

import logging
from os.path import join
from gensim.models import Word2Vec
import numpy as np
import random
from utils.cache import LMDBClient
from utils import data_utils
from utils.data_utils import Singleton
from utils import settings

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
EMB_DIM = 100



# class MySenteces():
#     def __init__(self, dirname):
#         self.dirname = dirname

#     def __iter__(self):
#     for fname in os.listdir(self.dirname):
#         for line in open(os.path.join(self.dirname, fname)):
#             yield line.split()
            
class MySenteces1():
    def __init__(self, dirname):
        self.LMDB_NAME = dirname
        self.lc = LMDBClient(self.LMDB_NAME)
        

    def __iter__(self):
        with self.lc.db.begin() as txn:
            for k in txn.cursor():
                author_feature = data_utils.deserialize_embedding(k[1])
                random.shuffle(author_feature)
                yield author_feature

class Word2vecModel:

    def __init__(self, name="sci_alldata"):
        self.model = None
        self.name = name


    def train(self, wf_name, size=EMB_DIM):
        data = MySenteces1('sci_all_data_feature')
        self.model = Word2Vec(data,size=100,negative =5, min_count=2, window=5,workers=16)
        self.model.save(join(settings.EMB_DATA_DIR, '{}.emb'.format(wf_name)))


    def load(self, name):
        self.model = Word2Vec.load(join(settings.EMB_DATA_DIR, '{}.emb'.format(name)))
        return self.model

    def project_embedding(self, tokens, idf=None):
        """
        weighted average of token embeddings
        :param tokens: input words
        :param idf: IDF dictionary
        :return: obtained weighted-average embedding
        """
        if self.model is None:
            self.load(self.name)
            print('{} embedding model loaded'.format(self.name))
        vectors = []
        sum_weight = 0
        for token in tokens:
            if not token in self.model.wv:
                continue
            weight = 1
            if idf and token in idf:
                weight = idf[token]
            v = self.model.wv[token] * weight
            vectors.append(v)
            sum_weight += weight
        if len(vectors) == 0:
            print('all tokens not in w2v models')
            # return np.zeros(self.model.vector_size)
            return None
        emb = np.sum(vectors, axis=0)
        emb /= sum_weight
        return emb


if __name__ == '__main__':
    wf_name = 'sci_alldata'
    emb_model = EmbeddingModel()
    emb_model.train(wf_name)
    print('loaded')

2020-03-27 15:48:49,891 : INFO : collecting all words and their counts
2020-03-27 15:48:49,896 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-03-27 15:48:50,196 : INFO : PROGRESS: at sentence #10000, processed 273128 words, keeping 23934 word types
2020-03-27 15:48:50,563 : INFO : PROGRESS: at sentence #20000, processed 585527 words, keeping 37515 word types
2020-03-27 15:48:51,290 : INFO : PROGRESS: at sentence #30000, processed 1254025 words, keeping 60470 word types
2020-03-27 15:48:52,191 : INFO : PROGRESS: at sentence #40000, processed 2065677 words, keeping 75355 word types
2020-03-27 15:48:52,631 : INFO : PROGRESS: at sentence #50000, processed 2418091 words, keeping 83152 word types
2020-03-27 15:48:53,332 : INFO : PROGRESS: at sentence #60000, processed 3043595 words, keeping 93118 word types
2020-03-27 15:48:53,580 : INFO : PROGRESS: at sentence #70000, processed 3260036 words, keeping 95888 word types
2020-03-27 15:48:54,071 : INFO : PROGRESS

2020-03-27 15:50:03,283 : INFO : PROGRESS: at sentence #710000, processed 61489731 words, keeping 550488 word types
2020-03-27 15:50:04,700 : INFO : PROGRESS: at sentence #720000, processed 62663889 words, keeping 556523 word types
2020-03-27 15:50:05,827 : INFO : PROGRESS: at sentence #730000, processed 63689129 words, keeping 562039 word types
2020-03-27 15:50:07,026 : INFO : PROGRESS: at sentence #740000, processed 64771864 words, keeping 567783 word types
2020-03-27 15:50:08,306 : INFO : PROGRESS: at sentence #750000, processed 65891761 words, keeping 573568 word types
2020-03-27 15:50:09,751 : INFO : PROGRESS: at sentence #760000, processed 67072203 words, keeping 580239 word types
2020-03-27 15:50:11,131 : INFO : PROGRESS: at sentence #770000, processed 68111191 words, keeping 585564 word types
2020-03-27 15:50:12,674 : INFO : PROGRESS: at sentence #780000, processed 69325794 words, keeping 592833 word types
2020-03-27 15:50:13,961 : INFO : PROGRESS: at sentence #790000, processe

2020-03-27 15:51:32,183 : INFO : PROGRESS: at sentence #1410000, processed 137775515 words, keeping 896523 word types
2020-03-27 15:51:33,353 : INFO : PROGRESS: at sentence #1420000, processed 138795730 words, keeping 900392 word types
2020-03-27 15:51:34,578 : INFO : PROGRESS: at sentence #1430000, processed 139909487 words, keeping 904411 word types
2020-03-27 15:51:35,815 : INFO : PROGRESS: at sentence #1440000, processed 141035312 words, keeping 908588 word types
2020-03-27 15:51:36,941 : INFO : PROGRESS: at sentence #1450000, processed 142071174 words, keeping 913071 word types
2020-03-27 15:51:38,089 : INFO : PROGRESS: at sentence #1460000, processed 143121615 words, keeping 917446 word types
2020-03-27 15:51:39,092 : INFO : PROGRESS: at sentence #1470000, processed 144033356 words, keeping 921051 word types
2020-03-27 15:51:40,370 : INFO : PROGRESS: at sentence #1480000, processed 145192391 words, keeping 925539 word types
2020-03-27 15:51:41,556 : INFO : PROGRESS: at sentence #

2020-03-27 15:53:01,937 : INFO : PROGRESS: at sentence #2110000, processed 217041947 words, keeping 1175506 word types
2020-03-27 15:53:03,041 : INFO : PROGRESS: at sentence #2120000, processed 217867786 words, keeping 1178037 word types
2020-03-27 15:53:04,270 : INFO : PROGRESS: at sentence #2130000, processed 218925931 words, keeping 1181240 word types
2020-03-27 15:53:05,716 : INFO : PROGRESS: at sentence #2140000, processed 220107449 words, keeping 1184489 word types
2020-03-27 15:53:07,070 : INFO : PROGRESS: at sentence #2150000, processed 221194450 words, keeping 1188209 word types
2020-03-27 15:53:08,216 : INFO : PROGRESS: at sentence #2160000, processed 222090717 words, keeping 1192230 word types
2020-03-27 15:53:09,829 : INFO : PROGRESS: at sentence #2170000, processed 223346280 words, keeping 1196627 word types
2020-03-27 15:53:11,371 : INFO : PROGRESS: at sentence #2180000, processed 224571139 words, keeping 1200602 word types
2020-03-27 15:53:12,218 : INFO : PROGRESS: at se

2020-03-27 15:54:32,088 : INFO : PROGRESS: at sentence #2800000, processed 292648869 words, keeping 1397394 word types
2020-03-27 15:54:33,510 : INFO : PROGRESS: at sentence #2810000, processed 293737109 words, keeping 1400306 word types
2020-03-27 15:54:34,610 : INFO : PROGRESS: at sentence #2820000, processed 294731972 words, keeping 1403939 word types
2020-03-27 15:54:35,920 : INFO : PROGRESS: at sentence #2830000, processed 295914964 words, keeping 1407209 word types
2020-03-27 15:54:36,910 : INFO : PROGRESS: at sentence #2840000, processed 296842835 words, keeping 1409771 word types
2020-03-27 15:54:37,958 : INFO : PROGRESS: at sentence #2850000, processed 297808092 words, keeping 1412826 word types
2020-03-27 15:54:39,195 : INFO : PROGRESS: at sentence #2860000, processed 298939160 words, keeping 1416488 word types
2020-03-27 15:54:40,553 : INFO : PROGRESS: at sentence #2870000, processed 300129252 words, keeping 1419729 word types
2020-03-27 15:54:41,955 : INFO : PROGRESS: at se

...

2020-03-27 22:47:55,203 : INFO : EPOCH 5 - PROGRESS: at 96.12% examples, 506461 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:47:56,212 : INFO : EPOCH 5 - PROGRESS: at 96.14% examples, 506472 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:47:57,214 : INFO : EPOCH 5 - PROGRESS: at 96.15% examples, 506487 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:47:58,222 : INFO : EPOCH 5 - PROGRESS: at 96.16% examples, 506497 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:47:59,224 : INFO : EPOCH 5 - PROGRESS: at 96.17% examples, 506502 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:48:00,225 : INFO : EPOCH 5 - PROGRESS: at 96.19% examples, 506514 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:48:01,238 : INFO : EPOCH 5 - PROGRESS: at 96.20% examples, 506528 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:48:02,247 : INFO : EPOCH 5 - PROGRESS: at 96.22% examples, 506534 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:48:03,269 : INFO : EPOCH 5 - PROGRESS: at 96.23% examples, 506537 words/s, in_qsiz

2020-03-27 22:49:08,839 : INFO : EPOCH 5 - PROGRESS: at 97.12% examples, 507212 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:49:09,842 : INFO : EPOCH 5 - PROGRESS: at 97.13% examples, 507220 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:49:10,851 : INFO : EPOCH 5 - PROGRESS: at 97.15% examples, 507228 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:49:11,854 : INFO : EPOCH 5 - PROGRESS: at 97.17% examples, 507247 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:49:12,869 : INFO : EPOCH 5 - PROGRESS: at 97.18% examples, 507259 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:49:13,871 : INFO : EPOCH 5 - PROGRESS: at 97.25% examples, 507261 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:49:14,876 : INFO : EPOCH 5 - PROGRESS: at 97.26% examples, 507274 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:49:15,879 : INFO : EPOCH 5 - PROGRESS: at 97.28% examples, 507286 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:49:16,883 : INFO : EPOCH 5 - PROGRESS: at 97.29% examples, 507298 words/s, in_qsiz

2020-03-27 22:50:23,316 : INFO : EPOCH 5 - PROGRESS: at 98.15% examples, 507598 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:50:24,330 : INFO : EPOCH 5 - PROGRESS: at 98.16% examples, 507597 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:50:25,336 : INFO : EPOCH 5 - PROGRESS: at 98.18% examples, 507602 words/s, in_qsize 0, out_qsize 1
2020-03-27 22:50:26,342 : INFO : EPOCH 5 - PROGRESS: at 98.19% examples, 507615 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:50:27,357 : INFO : EPOCH 5 - PROGRESS: at 98.21% examples, 507621 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:50:28,378 : INFO : EPOCH 5 - PROGRESS: at 98.22% examples, 507632 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:50:29,404 : INFO : EPOCH 5 - PROGRESS: at 98.23% examples, 507641 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:50:30,406 : INFO : EPOCH 5 - PROGRESS: at 98.24% examples, 507645 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:50:31,416 : INFO : EPOCH 5 - PROGRESS: at 98.26% examples, 507656 words/s, in_qsiz

2020-03-27 22:51:37,148 : INFO : EPOCH 5 - PROGRESS: at 99.20% examples, 508226 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:51:38,173 : INFO : EPOCH 5 - PROGRESS: at 99.21% examples, 508231 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:51:39,176 : INFO : EPOCH 5 - PROGRESS: at 99.25% examples, 508230 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:51:40,193 : INFO : EPOCH 5 - PROGRESS: at 99.27% examples, 508231 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:51:41,202 : INFO : EPOCH 5 - PROGRESS: at 99.30% examples, 508226 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:51:42,208 : INFO : EPOCH 5 - PROGRESS: at 99.32% examples, 508224 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:51:43,212 : INFO : EPOCH 5 - PROGRESS: at 99.35% examples, 508223 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:51:44,222 : INFO : EPOCH 5 - PROGRESS: at 99.39% examples, 508219 words/s, in_qsize 0, out_qsize 0
2020-03-27 22:51:45,237 : INFO : EPOCH 5 - PROGRESS: at 99.43% examples, 508212 words/s, in_qsiz

loaded
